# COCO128 + Detectron2 + 3LC Tutorial

This notebook focuses on metrics collection for object detection using the Detectron2 library and the COCO128 subset. No
training is performed; instead, we use a pretrained model to evaluate performance. Metrics related to bounding boxes
(true positives, false positives, false negatives, iou, confidence) are collected using the BoundingBoxMetricsCollector
class.

The notebook illustrates:

+ Metrics collection on a pretrained Detectron2 model using the COCO128 subset.
+ Using `BoundingBoxMetricsCollector` for collecting object detection metrics.

In [ ]:
# Parameters
PROJECT_NAME = "COCO128"
TRAIN_DATASET_NAME = "COCO128"
TRANSIENT_DATA_PATH = "./transient_data"
TEST_DATA_PATH = "../tests/test_data/data"

# Hyperparameters
MODEL_CONFIG = "COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"
MAX_DETECTIONS_PER_IMAGE = 30
SCORE_THRESH_TEST = 0.5
INSTALL_DEPENDENCIES = False
TLC_PUBLIC_EXAMPLES_DEVELOPER_MODE = True

In [ ]:
%%capture
if INSTALL_DEPENDENCIES:
    %pip install torch==1.10.1+cpu torchvision==0.11.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
    %pip install tlc[detectron2]
    %pip install opencv-python
    %pip install matplotlib

In [ ]:
import tlc
import torch
import detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("tlc: ", tlc.__version__)
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
from __future__ import annotations

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
logger = setup_logger()
logger.setLevel("ERROR")

# import some common libraries
import cv2
import random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


## Prepare the dataset

A small subset of the original COCO dataset (in the COCO standard format) is available in the `../data/coco128` directory.

## Register the dataset with 3LC

Now that we have the dataset in the COCO format, we can register it with 3LC.

In [ ]:
from tlc.integration.detectron2 import register_coco_instances

register_coco_instances(
    TRAIN_DATASET_NAME,
    {},
    train_json_path.to_str(),
    train_image_folder.to_str(),
    project_name=PROJECT_NAME,
)

In [ ]:
# The detectron2 dataset dicts and dataset metadata can be read from the DatasetCatalog and MetadataCatalog, respectively.
dataset_metadata = MetadataCatalog.get(TRAIN_DATASET_NAME)
dataset_dicts = DatasetCatalog.get(TRAIN_DATASET_NAME)

To verify the dataset is in correct format, let's visualize the annotations of randomly selected samples in the training set:

In [ ]:
from detectron2.utils.file_io import PathManager
import numpy as np

for d in random.sample(dataset_dicts, 3):
    filename = tlc.Url(d["file_name"]).to_absolute().to_str()
    if "s3://" in filename:
        with PathManager.open(filename, "rb") as f:
            img = np.asarray(bytearray(f.read()), dtype="uint8")
            img = cv2.imdecode(img, cv2.IMREAD_COLOR)
    else:
        img = cv2.imread(filename)
    visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    out_rgb = cv2.cvtColor(out.get_image(), cv2.COLOR_BGR2RGB)
    plt.imshow(out_rgb[:, :, ::-1])
    plt.title(filename.split("/")[-1])
    plt.show()

## Start a 3LC Run and collect bounding box evaluation metrics


In [ ]:
run = tlc.init(PROJECT_NAME)

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file(MODEL_CONFIG))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(MODEL_CONFIG)
cfg.DATASETS.TRAIN = (TRAIN_DATASET_NAME,)
cfg.OUTPUT_DIR = TRANSIENT_DATA_PATH
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.TEST.DETECTIONS_PER_IMAGE = MAX_DETECTIONS_PER_IMAGE
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = SCORE_THRESH_TEST
cfg.MODEL.DEVICE = "cpu"
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

config = {
    "model_config": MODEL_CONFIG,
    "test.detections_per_image": MAX_DETECTIONS_PER_IMAGE,
    "model.roi_heads.score_thresh_test": SCORE_THRESH_TEST,
}

run.set_parameters(config)

In [ ]:
from detectron2.engine import DefaultTrainer
from tlc.integration.detectron2 import MetricsCollectionHook

trainer = DefaultTrainer(cfg)

metrics_collector = tlc.BoundingBoxMetricsCollector(
                model=trainer.model,
                classes=dataset_metadata.thing_classes,
                label_mapping=dataset_metadata.thing_dataset_id_to_contiguous_id,
                iou_threshold=0.5,
                compute_derived_metrics=True
)

metrics_collection_hook = MetricsCollectionHook(
                dataset_name=TRAIN_DATASET_NAME,
                metrics_collectors=[metrics_collector],
                collect_metrics_before_train=True,
            )

trainer.register_hooks([metrics_collection_hook])
trainer.resume_or_load(resume=False)
trainer.before_train()